# The Price is Right

It is a project for ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [15:16<00:00, 183.28s/it]


In [5]:
len(deals)

50

In [6]:
deals[44].describe()

"Title: LG Major Appliance Special Values at Lowe's: Up to 50% off + Extra 10% off select $396+ + Pickup or $29 delivery\nDetails: Shop over 200 major appliances from LG. Select items priced $396 or more get an extra 10% off in cart. We've pictured the LG 4.5-cu ft High Efficiency Stackable Front-Load Washer for $647.99 ($251 off). Buy Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/LG-Major-Appliance-Special-Values-at-Lowes-Up-to-50-off-Extra-10-off-select-396-Pickup-or-29-delivery/21694120.html?iref=rss-c196"

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Best Buy Black Friday TV Sale: Up to 45% off + free shipping
Details: Big-brand 50" smart TVs start at $250, 75" models at $850, and 85" at $1,300. Pictured is the Samsung Q70D Series QN65Q70DAFXZA 75" TV, which is $800 (a $200 drop). Shop Now at Best Buy
Features: 
URL: https://www.dealnews.com/Best-Buy-Black-Friday-TV-Sale-Up-to-45-off-free-shipping/21694160.html?

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [21]:
len(result.deals)

5

In [22]:
result.deals[1]

Deal(product_description='The Hisense A7 Series 65A7N 65" Class 4K UHD Smart Google TV offers a stunning 3840x2160 native resolution for a crystal-clear viewing experience. Built-in Google Assistant and Amazon Alexa support offer seamless voice control, and you have access to popular streaming services like Netflix, Disney+, and YouTube directly from the TV. The device features WiFi connectivity and includes three HDMI inputs, making it easy to connect multiple devices. This model is designed for a smart home experience with modern technology.', price=350.0, url='https://www.dealnews.com/products/Hisense/Hisense-A7-Series-65-A7-N-65-Class-4-K-UHD-Smart-Google-TV/471574.html?iref=rss-c142')

In [23]:
from agents.scanner_agent import ScannerAgent

In [24]:
agent = ScannerAgent()
result = agent.scan()

In [25]:
result

DealSelection(deals=[Deal(product_description='The Philips Hue White & Color Ambiance A19 LED Starter Kit is an innovative light solution that includes three smart LED bulbs. With the ability to produce 16 million colors and a timer sync feature for media, it seamlessly integrates into your smart home. This kit requires a Hue Bridge for automation, making it an ideal choice for enhancing your home’s ambiance. Model: 556704SKU.', price=110.0, url='https://www.dealnews.com/products/Philips/Philips-Hue-White-Color-Ambiance-A19-LED-Starter-Kit/375866.html?iref=rss-f1912'), Deal(product_description='The Hisense A7 Series 65A7N is a 65-inch Class 4K UHD Smart Google TV, offering a stunning 3840x2160 native resolution. With built-in Google Assistant and Amazon Alexa compatibility, it provides seamless access to streaming services like Netflix, Disney+, and YouTube. Equipped with WiFi connectivity and three HDMI inputs, this smart TV is perfect for a modern entertainment experience.', price=35

In [26]:
print("test")

test
